## 3.2 找出 text_1 == text_2 的句子对

In [4]:
# 查询带有指定字词的句子对
def compare_equal_text_pair(text_list_1, text_list_2, text_label_list, print_progress = False):
    text_pairs_list = []
    label_list = []
    print("="*15 + " In Progress " + "="*15)
    for i in range (0, len(text_list_1)):
        text_1 = text_list_1[i]
        text_2 = text_list_2[i]
        
        if text_1 == text_2:
            text_pairs_list.append(text_list_1[i] + "\t" + text_list_2[i])
            if text_label_list!= None:
                label_list.append(str(i) + "\t" + str(text_label_list[i]))
 
        if print_progress and i%10000 == 0:
            print("%.2f of text pairs for different subsets identification have processed " %((i/len(text_list_1))*100))    
    print("="*15 + "     Done    " + "="*15)
    return text_pairs_list, label_list

# 使用 准备数据
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ccf_qianyan_qm_result_B.csv")

# 进行识别
equal_text_pairs_list, equal_label_list = compare_equal_text_pair(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
print(len(equal_text_pairs_list))
print(equal_text_pairs_list[0])
print(len(equal_label_list))
print(equal_label_list[0])
write_file("common_data/equal_1_list.txt",equal_text_pairs_list)
print(len(read_candidates_line("common_data/equal_1_list.txt")))

# 打印结果
test_zero_list, test_one_list = summarize_one_zero(equal_text_pairs_list, equal_label_list)
print("\none in homophone_text_pairs_1_list accounts for %.2f" %(len(test_one_list)/len(equal_text_pairs_list)*100))
print(len(test_one_list))
print_examples(test_one_list, None, None, 50)
print(len(test_zero_list))
print_examples(test_zero_list, None, None, 50)



The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ccf_qianyan_qm_result_B.csv is 100000
=============== In Progress ===============
0.00 of text pairs for different subsets identification have processed 
10.00 of text pairs for different subsets identification have processed 
20.00 of text pairs for different subsets identification have processed 
30.00 of text pairs for different subsets identification have processed 
40.00 of text pairs for different subsets identification have processed 
50.00 of text pairs for different subsets identification have processed 
60.00 of text pairs for different subsets identification have processed 
70.00 of text pairs for different subsets identification have processed 
80.00 of text pairs for different subsets identification have processed 
90.00 of text pairs for different subsets identification have processed 
===============     Done    ===============
4116
萧红带我回到身后的村庄的启示	萧红带我回到身后的村

## 3.3 检测仅有同音字不同的句子对（lazypinyin）

In [7]:
# 检测（1）对比两个句子是否拼音一致但书写不一致（2）有同音字不同的句子对（lazypinyin） 
from pypinyin import lazy_pinyin
def compare_pinyin_and_text(text_list_1, text_list_2, text_label_list, print_progress = False):
    special_pinyin_list = [["b","p"],["m","n"],["zh","z"],["ch","c"],["sh","s"],["r","l"],["r","n"],["n","l"],["en","eng"],["an","ang"],["in","ing"],["v","i"],["v","u"],["u","i"]]
    special_words_list =[["四","十"],['汉','韩']]
    special_sentence_list = ["那里人","哪里人","哪人","多少岁"]
    pinyin_0_list = []
    label_0_list = []
    pinyin_1_list = []
    label_1_list = []
    count_percent = 0
    print("="*15 + " In Progress " + "="*15)
    for i in range (0, len(text_list_1)):
        count_percent = count_percent + 1
        
        text_1 = text_list_1[i]
        text_2 = text_list_2[i]

        subset_1, subset_2 = check_subset(text_1, text_2)

        contain_special_words = False
        if special_words_list != []:
            for special_word in special_words_list:
                if (special_word[0] == subset_1 and special_word[1] == subset_2) or (special_word[0] == subset_2 and special_word[1] == subset_1):
                    pinyin_0_list.append(text_list_1[i] + "\t" + text_list_2[i])
                    if text_label_list!= None:
                        label_0_list.append(str(i) + "\t" + str(text_label_list[i]))
                    contain_special_words = True
        contain_special_sentences = False
        if special_sentence_list != []:
            for special_sentence in special_sentence_list:
                if special_sentence in text_1 and special_sentence in text_2:
                    contain_special_sentences = True
        # 整句话的拼音是否一致
        if not contain_special_words and lazy_pinyin(text_1) == lazy_pinyin(text_2) and text_1 != text_2:
            if contain_special_sentences:
                pinyin_0_list.append(text_list_1[i] + "\t" + text_list_2[i])
                if text_label_list!= None:
                    label_0_list.append(str(i) + "\t" + str(text_label_list[i]))
            else:
                pinyin_1_list.append(text_list_1[i] + "\t" + text_list_2[i])
                if text_label_list!= None:
                    label_1_list.append(str(i) + "\t" + str(text_label_list[i]))
        # 分析两句同样长度的话中不一样的部分拼音是否一致（因为整句话的识别 哪 nei 那 na等）
        elif not contain_special_words and subset_1!="Not_The_Case" and len(subset_1) > 0 and len(subset_1) == len(subset_2):
            x = lazy_pinyin(subset_1)
            y = lazy_pinyin(subset_2)
            if x == y:
                if contain_special_sentences:
                    pinyin_0_list.append(text_list_1[i] + "\t" + text_list_2[i])
                    if text_label_list!= None:
                        label_0_list.append(str(i) + "\t" + str(text_label_list[i]))
                else:
                    pinyin_1_list.append(text_list_1[i] + "\t" + text_list_2[i])
                    if text_label_list!= None:
                        label_1_list.append(str(i) + "\t" + str(text_label_list[i]))
            elif len(x) == len (y) and x != y:
                for special_pinyin in special_pinyin_list:
                    for index in range(0,len(x)):
                        if special_pinyin[0] in x[index] and special_pinyin[0] not in y[index]:
                            x[index] = x[index].replace(special_pinyin[0],special_pinyin[1])
                        elif special_pinyin[0] in y[index] and special_pinyin[0] not in x[index]:
                            y[index] = y[index].replace(special_pinyin[0],special_pinyin[1])
                    if x == y :
                        if contain_special_sentences:
                            pinyin_0_list.append(text_list_1[i] + "\t" + text_list_2[i])
                            if text_label_list!= None:
                                label_0_list.append(str(i) + "\t" + str(text_label_list[i]))
                        else:
                            pinyin_1_list.append(text_list_1[i] + "\t" + text_list_2[i])
                            if text_label_list!= None:
                                label_1_list.append(str(i) + "\t" + str(text_label_list[i]))
        
        if print_progress and count_percent%10000 == 0:
            print("%.2f of text pairs for different subsets identification have processed " %((count_percent/len(text_list_1))*100))    
    print("="*15 + "     Done    " + "="*15)
    return pinyin_1_list, label_1_list, pinyin_0_list, label_0_list

# 使用 准备数据
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ccf_qianyan_qm_result_B.csv")

# 进行识别
homophone_text_pairs_1_list, homophone_label_1_list, homophone_text_pairs_0_list, homophone_label_0_list = compare_pinyin_and_text(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
print(len(homophone_text_pairs_1_list))
print(homophone_text_pairs_1_list[0])
print(len(homophone_label_1_list))
print(homophone_label_1_list[0])
write_file("common_data/pinyin_1_text.txt",homophone_text_pairs_1_list)
write_file("common_data/pinyin_0_text.txt",homophone_text_pairs_0_list)

# 使用 方法2-7 统计测试集中，预测结果同音字的0/1概率
test_zero_list, test_one_list = summarize_one_zero(homophone_text_pairs_1_list, homophone_label_1_list)
print("\none in homophone_text_pairs_1_list accounts for %.2f" %(len(test_one_list)/len(homophone_text_pairs_1_list)*100))
print("zero in homophone_text_pairs_1_list accounts for %.2f" %(len(test_zero_list)/len(homophone_text_pairs_1_list)*100))
print(len(test_one_list))
print_examples(test_one_list, None, None, 50)
print(len(test_zero_list))
print_examples(test_zero_list, None, None, 50)

test_zero_list, test_one_list = summarize_one_zero(homophone_text_pairs_0_list, homophone_label_0_list)
if homophone_text_pairs_0_list != []:
    print("\none in homophone_text_pairs_0_list accounts for %.2f" %(len(test_one_list)/len(homophone_text_pairs_0_list)*100))
    print("zero in homophone_text_pairs_0_list accounts for %.2f" %(len(test_zero_list)/len(homophone_text_pairs_0_list)*100))
print(len(test_one_list))
print_examples(test_one_list, None, None, 50)
print(len(test_zero_list))
print_examples(test_zero_list, None, None, 50)


The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ccf_qianyan_qm_result_B.csv is 100000
=============== In Progress ===============
10.00 of text pairs for different subsets identification have processed 
20.00 of text pairs for different subsets identification have processed 
30.00 of text pairs for different subsets identification have processed 
40.00 of text pairs for different subsets identification have processed 
50.00 of text pairs for different subsets identification have processed 
60.00 of text pairs for different subsets identification have processed 
70.00 of text pairs for different subsets identification have processed 
80.00 of text pairs for different subsets identification have processed 
90.00 of text pairs for different subsets identification have processed 
100.00 of text pairs for different subsets identification have processed 
===============     Done    ===============
3349
小孩补什么钙好	小孩补什么盖好
3349
40

## 3.4 检测有错别字的句子对（pycorrector）

In [2]:
pip install pycorrector

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 检测有错别字的句子对（pycorrector）
# import pycorrector
from pycorrector.ernie.ernie_corrector import ErnieCorrector

m = ErnieCorrector()

def identify_misspelling_pycorrector(text_list_1, text_list_2, text_label_list, print_progress = False):
    text_pairs_list = []
    subset_list = []
    label_list = []
    count_percent = 0
    print("="*15 + " In Progress " + "="*15)
    for i in range (0, len(text_list_1)):
        count_percent = count_percent + 1
        
        text_1 = text_list_1[i]
        text_2 = text_list_2[i]

        if text_1!=text_2 and len(text_1) == len(text_1):
            corrected_sent_1, detail_1 = m.ernie_correct(text_1)
            corrected_sent_2, detail_2 = m.ernie_correct(text_2)
            if corrected_sent_1 == corrected_sent_2:
                text_pairs_list.append(text_list_1[i] + "\t" + text_list_2[i])
                if text_label_list!= None:
                    label_list.append(str(i) + "\t" + str(text_label_list[i]))      
        if print_progress and count_percent%5000 == 0:
            print("%.2f of text pairs for pycorrect identification have processed " %((count_percent/len(text_list_1))*100))    
    print("="*15 + "     Done    " + "="*15)
    return text_pairs_list, label_list

# 使用 准备数据
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ccf_qianyan_qm_result_B.csv")

# 进行识别
pycorrect_text_pairs_list, pycorrect_label_list = identify_misspelling_pycorrector(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
print(len(pycorrect_text_pairs_list))
print(pycorrect_text_pairs_list[0])
print(len(pycorrect_label_list))
print(pycorrect_label_list[0])
write_file("common_data/pycorrector_1_list.txt",pycorrect_text_pairs_list)
print(len(read_candidates_line("common_data/pycorrector_1_list.txt")))

# 统计测试集中，预测结果同音字的0/1概率
test_zero_list, test_one_list = summarize_one_zero(pycorrect_text_pairs_list, pycorrect_label_list)
print("\none in pycorrect_text_pairs_list accounts for %.2f" %(len(test_one_list)/len(pycorrect_text_pairs_list)*100))
print("zero in pycorrect_text_pairs_list accounts for %.2f" %(len(test_zero_list)/len(pycorrect_text_pairs_list)*100))
print(len(test_one_list))
print_examples(test_one_list, None, None, 5)
print(len(test_zero_list))
print_examples(test_zero_list, None, None, 5)

ModuleNotFoundError: No module named 'torch'

## 3.5 检测有错别字的句子对(去除部首后对比）

In [15]:
# 判断一个的笔画是否包含在另一个字的笔画中
def check_word_contain(a, b, bihua_1, bushou_1, bihua_2, bushou_2):
    is_contain = True
    long_bihua = []
    short_bihua = []
    different_bihua = []
    long_bushou = []
    # 对比两个字的笔画，将笔画多的放入long_bihua其部首放入long_bushou，少的放入short_bihua,
    if len(bihua_1) > len(bihua_2):
        long_bihua = bihua_1
        short_bihua = bihua_2
        long_bushou = bushou_1
    else:
        long_bihua = bihua_2
        short_bihua = bihua_1
        long_bushou = bushou_2
    
    # 情况1 如果这两个字的从第一笔开始到len（short_bihua）存在不同，则为 不包含情况1
    for i in range(0, len(short_bihua)):
        if short_bihua[i] != long_bihua[i]:
            is_contain = False
    
    # 如果这两个字的从第一笔开始到len（short_bihua）都相同，则为 包含 
    if is_contain == True:
        # 判断剩下的笔画是否为多笔画字的部首
        different_bihua = long_bihua[len(short_bihua):len(long_bihua)]
        # 若不同部分仅为部首，返回True。否则False
        if different_bihua == long_bushou:
            return True
        else:
            return False
    else:
        is_contain = True
        # 情况2 如果这两个字的从len（short_bihua） 到 最后 存在不同，则为 不包含情况2
        for i in range(0, len(short_bihua)):
            if short_bihua[i] != long_bihua[i + len(long_bihua) - len(short_bihua)]:
                is_contain = False
        # 如果这两个字的从len（short_bihua） 到 最后 都相同，则为 包含 
        if is_contain == True:
            # 判断剩下的笔画是否为多笔画字的部首
            different_bihua = long_bihua[0:len(long_bihua) - len(short_bihua)]
            # 若不同部分仅为部首，返回True。否则False
            if different_bihua == long_bushou:
                return True
            else:
                return False
        else:
            return False

In [16]:
# 去掉两个字符串中相同的连续部分
def remove_bushou(a, b, text_1, bushou_1, text_2, bushou_2):

    if text_1!= bushou_1 and text_2 != bushou_2:
        #获取除了部首之外的笔画
        rest_1 = []
        rest_2 = []

        # print("text_1: " + str(text_1))
        # print("bushou_1:"  + str(bushou_1))
        # print("text_2: " + str(text_2))
        # print("bushou_2: " + str(bushou_2))

        subset_1_1, subset_1_2 = check_subset(text_1, bushou_1)
        subset_2_1, subset_2_2 = check_subset(text_2, bushou_2)
        
        # print(subset_1_1,subset_1_2 )
        # print(subset_2_1,subset_2_2 )
        # 第一个字去掉部首后剩余的笔画
        if subset_1_1 == "Not_The_Case" or subset_1_2 == "Not_The_Case":
            x = 1
        elif subset_1_2 ==[] and subset_1_1!=[]: 
            rest_1 = subset_1_1 
        elif subset_1_2 !=[] and subset_1_1==[]:
            rest_1 = subset_1_2

        # 第二个字去掉部首后剩余的笔画
        if subset_2_1 == "Not_The_Case" or subset_2_2 == "Not_The_Case":
            x = 1
        elif subset_2_2 ==[] and subset_2_1!=[]: 
            rest_2 = subset_2_1 
        elif subset_2_2 !=[] and subset_2_1==[]:
            rest_2 = subset_2_2

        
        # print(rest_1,rest_2)  
        #若剩余比划相同 且 不为空
        if rest_1 == rest_2 and rest_1 != []:
            return True
        else:
            return False
    else:
        # 若分解出的部首就是该字本身
        return False


In [17]:
#通过cixing来分解汉字部首和笔画
from cixing.cixing_5 import *
def compare_bushou_bihua(text_1,text_2):
    handler = ChineseCixing()
    #获得全字的笔画
    strokes_original_1 = handler.get_strokes(text_1)
    strokes_original_2 = handler.get_strokes(text_2)
    #获得全字的部首
    radicals_original_1 = handler.get_radical(text_1)
    radicals_original_2 = handler.get_radical(text_2)
    #获得部首的笔画
    radicals_bushou_1 = handler.get_strokes(radicals_original_1)
    radicals_bushou_2 = handler.get_strokes(radicals_original_2)
    #如果其中一个字是另一个字的一部分，如 不/怀 返回True
    if check_word_contain(text_1, text_2, strokes_original_1[0],radicals_bushou_1[0],strokes_original_2[0],radicals_bushou_2[0]) :
        # print("其中一个字是另一个字去掉部首后的部分")
        return True
    else:
        # 部首不同 如 怀/还
        if radicals_original_1[0] != radicals_original_2[0]:
            # print("部首不同")
            comparison = remove_bushou(text_1, text_2, strokes_original_1[0],radicals_bushou_1[0],strokes_original_2[0],radicals_bushou_2[0])
            return comparison
        else:
            # print("部首相同")
            return False

In [18]:
# 检测仅一字不同，且除部首外有相似结构
# 是否是中文
def is_Chinese(ch): 
    if '\u4e00' <= ch <= '\u9fff': 
        return True 
    return False

# 方法3-3 检测仅一字不同，且除部首外有相似结构
def identify_misspelling_structure(text_list_1, text_list_2, text_label_list, print_progress = False):
    text_pairs_list = []
    label_list = []
    print("="*15 + " In Progress " + "="*15)
    for i in range (0, len(text_list_1)):

        text_1 = text_list_1[i]
        text_2 = text_list_2[i]

        subset_1, subset_2 = check_subset(text_1, text_2)
        if subset_1!="Not_The_Case" and subset_1!="" and subset_2!="":
            # 如果有一个字不一样，尝试发现是否是结构上的错别字
            if len(subset_1) == len(subset_2) and len(subset_2) == 1 :
                if is_Chinese(subset_1) and is_Chinese(subset_2) and not(subset_1.isnumeric() or subset_2.isnumeric()):
                    has_similar_part = compare_bushou_bihua(subset_1,subset_2)
                    if has_similar_part:
                        text_pairs_list.append(text_list_1[i] + "\t" + text_list_2[i])
                        if text_label_list!= None:
                            label_list.append(str(i) + "\t" + str(text_label_list[i]))
        if print_progress and i!=0 and i%2000 == 0:
            print("%.2f %of text pairs for 1 word different have processed " %((i/len(text_list_1))*100))  
    print("="*15 + "     Done    " + "="*15)
    return text_pairs_list, label_list

In [19]:
# 使用 准备数据
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ccf_qianyan_qm_result_B.csv")

# 进行识别
structure_text_pairs_list, structure_label_list = identify_misspelling_structure(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
print(len(structure_text_pairs_list))
print(structure_text_pairs_list[0])
print(len(structure_label_list))
print(structure_label_list[0])

write_file("common_data/structure_1_list.txt",structure_text_pairs_list)

# 统计测试集中，预测结果同音字的0/1概率
test_zero_list, test_one_list = summarize_one_zero(structure_text_pairs_list, structure_label_list)
print("\none in structure_text_pairs_list accounts for %.2f" %(len(test_one_list)/len(structure_text_pairs_list)*100))
print("zero in structure_text_pairs_list accounts for %.2f" %(len(test_zero_list)/len(structure_text_pairs_list)*100))
print(len(test_one_list))
print_examples(test_one_list, None, None, 5)
print(len(test_zero_list))
print_examples(test_zero_list, None, None, 5)


The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ccf_qianyan_qm_result_B.csv is 100000
=============== In Progress ===============
2.00 of text pairs for 1 word different have processed 
4.00 of text pairs for 1 word different have processed 
6.00 of text pairs for 1 word different have processed 
8.00 of text pairs for 1 word different have processed 
10.00 of text pairs for 1 word different have processed 
12.00 of text pairs for 1 word different have processed 
14.00 of text pairs for 1 word different have processed 
16.00 of text pairs for 1 word different have processed 
18.00 of text pairs for 1 word different have processed 
20.00 of text pairs for 1 word different have processed 
22.00 of text pairs for 1 word different have processed 
24.00 of text pairs for 1 word different have processed 
26.00 of text pairs for 1 word different have processed 
28.00 of text pairs for 1 word different have processed 
30.00 of 

## 3.6 同音人名与同音地名不作为mispelling

In [20]:
# 尝试发现仅有人名是否仅有一个字的区别，若有则返回True
def discover_naming_mistake(name_1, name_2):
    different_words = []
    if len(name_1) == len(name_2) and len(name_2)>=2:
        for i in range(0, len(name_1)):
            if name_1[i] != name_2[i]:
                different_words.append(name_1[i])
                different_words.append(name_2[i])
    if len(different_words) == 2:
        return True
    else:
        return False

# 方法 3-4 寻找包含人名但仅差别一个字的句子对
from pypinyin import lazy_pinyin
def identify_name_misspelling(data_list_1, data_list_2, data_label_list = None, print_progress = False):
    celebrities_list = []
    name_1_text_pairs_list = []
    name_1_label_list = []
    name_0_text_pairs_list = []
    name_0_label_list = []
    print("="*15 + " In Progress " + "="*15)
    count = 0
    for i in range (0, len(data_list_1)):
        text_1 = data_list_1[i]
        text_2 = data_list_2[i]

        subset_1, subset_2 = check_subset(text_1,text_2)

        is_same_name_entity = False

        if subset_1!="Not_The_Case" and len(subset_1) == 1 and len(subset_1) == len(subset_2):
            words_1, natures_1 = get_words_and_words_nature(text_1)
            words_2, natures_2 = get_words_and_words_nature(text_2)
            # 有特殊姓名的处理方法
            if celebrities_list != []:
                if "PER" in natures_1 and "PER" in natures_2 and len(natures_1) == len(natures_2):
                    # print(subset_1,subset_2)
                    for x in range(0, len(natures_1)):
                        if (natures_1[x] == "PER" and natures_2[x] == "PER") and (words_1[x] != words_2[x]):
                            # print(words_1[x] , words_2[x])
                            for celebrity in celebrities_list:
                                condition_1 = bool((celebrity == words_1[x]) and discover_naming_mistake(celebrity, words_2[x]))
                                condition_2 = bool((celebrity == words_2[x]) and discover_naming_mistake(celebrity, words_1[x]))
                                condition_3 = bool((words_1[x] in celebrity) and (celebrity in words_2[x]))
                                condition_4 = bool((words_2[x] in celebrity) and (celebrity in words_1[x]))
                                if  condition_1 or condition_2 or condition_3 or condition_4:
                                    count = count + 1
                                    name_1_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                                    if data_label_list!= None:
                                        name_1_label_list.append(str(i) + "\t" + data_label_list[i])
                                        is_same_name_entity = True
                                    break
                    for x in range(0, len(natures_1)):
                        if (natures_1[x] == "PER" and natures_2[x] == "PER") and (words_1[x] != words_2[x]) and not is_same_name_entity:
                            name_0_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                            if data_label_list!= None:
                                name_0_label_list.append(str(i) + "\t" + data_label_list[i])
            else:
                if "PER" in natures_1 and "PER" in natures_2 and len(natures_1) == len(natures_2):
                    # print(subset_1,subset_2)
                    for x in range(0, len(natures_1)):
                        if (natures_1[x] == "PER" and natures_2[x] == "PER") and (words_1[x] != words_2[x]):
                            name_0_text_pairs_list.append(data_list_1[i] + "\t" + data_list_2[i] )
                            if data_label_list!= None:
                                name_0_label_list.append(str(i) + "\t" + data_label_list[i]) 
        if print_progress and i%10000 == 0:
            print("%.2f of text pairs for name misspelling identification have processed " %((i/len(data_list_1))*100))    
    print("="*15 + "     Done    " + "="*15)
    print("带有相关信息的名人名字拼写错误的有: " + str(count) + " 对\n")
    return name_1_text_pairs_list, name_1_label_list, name_0_text_pairs_list, name_0_label_list

# 使用 准备数据
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ccf_qianyan_qm_result_B.csv") #仅仅用来查看，不影响结果

# 进行识别
name_1_text_pairs_list, name_1_label_list, name_0_text_pairs_list, name_0_label_list = identify_name_misspelling(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)

write_file("common_data/name_1_list.txt",name_1_text_pairs_list)
write_file("common_data/name_0_list.txt",name_0_text_pairs_list)

# 统计测试集中，预测结果同音字的0/1概率

print("==============" + str(len(name_1_text_pairs_list)) + "==============")
test_zero_list, test_one_list = summarize_one_zero(name_1_text_pairs_list, name_1_label_list)
if len(name_1_text_pairs_list)!=0:
    print("\none in name_1_text_pairs_list accounts for %.2f" %(len(test_one_list)/len(name_1_text_pairs_list)*100))
    print("zero in name_1_text_pairs_list accounts for %.2f" %(len(test_zero_list)/len(name_1_text_pairs_list)*100))
print_examples(test_one_list, None, None, 50)
print_examples(test_zero_list, None, None, 50)

print("==============" + str(len(name_0_text_pairs_list)) + "==============")
test_zero_list, test_one_list = summarize_one_zero(name_0_text_pairs_list, name_0_label_list)
if len(name_0_text_pairs_list)!=0:
    print("\none in name_0_text_pairs_list accounts for %.2f" %(len(test_one_list)/len(name_0_text_pairs_list)*100))
    print("zero in name_0_text_pairs_list accounts for %.2f" %(len(test_zero_list)/len(name_0_text_pairs_list)*100))
print_examples(test_one_list, None, None, 50)
print_examples(test_zero_list, None, None, 50)


The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ccf_qianyan_qm_result_B.csv is 100000
=============== In Progress ===============
0.00 of text pairs for name misspelling identification have processed 
10.00 of text pairs for name misspelling identification have processed 
20.00 of text pairs for name misspelling identification have processed 
30.00 of text pairs for name misspelling identification have processed 
40.00 of text pairs for name misspelling identification have processed 
50.00 of text pairs for name misspelling identification have processed 
60.00 of text pairs for name misspelling identification have processed 
70.00 of text pairs for name misspelling identification have processed 
80.00 of text pairs for name misspelling identification have processed 
90.00 of text pairs for name misspelling identification have processed 
===============     Done    ===============
带有相关信息的名人名字拼写错误的有: 0 对

==============0=

In [23]:
# 同音地名处理（无需重复运行）
# 去除 地名后的"省","市","县","区","镇","乡","村"
def remove_p_c_a(location):
    p_c_t = [
        "拉祜族佤族布朗族傣族自治县","保安族东乡族撒拉族自治县","彝族哈尼族拉祜族自治县","傣族拉祜族佤族自治县","哈尼族彝族傣族自治县",
        "哈尼族彝族傣族自治县","苗族瑶族傣族自治县","彝族回族苗族自治县","满族蒙古族自治县","土家族苗族自治州","哈尼族彝族自治县",
        "布依族苗族自治州","蒙古族藏族自治州","哈尼族彝族自治州","傣族景颇族自治州","土家族苗族自治县","白族普米族自治县",
        "苗族土家族自治县","布依族苗族自治县","土家族苗族自治县","苗族布依族自治县","仡佬族苗族自治县","哈尼族彝族自治县",
        "独龙族怒族自治县","壮族苗族自治州","苗族侗族自治州","黎族苗族自治县","壮族瑶族自治县","藏族羌族自治州",
        "苗族侗族自治县","回族苗族自治县","彝族傣族自治县","彝族傣族自治县","彝族苗族自治县","傣族佤族自治县","柯尔克孜自治州",
        "苗族彝族自治县","回族彝族自治县","回族土族自治县","彝族回族自治县","族撒拉族自治县","哈萨克族自治县","哈萨克自治州",
        "拉祜族自治县","纳西族自治县","哈尼族自治县","傈僳族自治县","哈萨克自治县","仫佬族自治县","毛南族自治县",
        "傈僳族自治州","朝鲜族自治县","朝鲜族自治州","蒙古族自治县","撒拉族自治县","土家族自治县","维吾尔自治区",
        "各族自治县","佤族自治县","回族自治县","土族自治县","彝族自治县","锡伯自治县","水族自治县","怒族自治县","羌族自治县",
        "藏族自治县","彝族自治县","苗族自治县","侗族自治县","瑶族自治县","满族自治县","回族自治县","蒙古自治州","白族自治州",
        "固族自治县","瑶族自治县","傣族自治州","苗族自治县","藏族自治州","畲族自治县","彝族自治州","回族自治州","壮族自治区",
        "黎族自治县","锡伯自治县","特别行政区","地区","自治区","自治县","省","市","盟","县","区","镇","乡","村"]
    special_location = ["神农架林区"]
    for x in p_c_t:
        if location == "神农架林区":
            location = "神农架"
        elif len(location) > 2 and x in location:
            location = location[:-len(x)]
            break
    return location
# 读取地名数据
import json
def read_location(json_file_path):
    with open(json_file_path, 'r') as file:
        china_location_data = json.load(file)
    locations_province_list = []
    locations_city_list = []
    locations_area_list = []
    
    for p_i in range(0, len(china_location_data)):
        province = remove_p_c_a(china_location_data[p_i]['name'])
        locations_province_list.append(province)
        for c_i in range(0, len(china_location_data[p_i]['city'])):
            city = remove_p_c_a(china_location_data[p_i]['city'][c_i]['name'])
            locations_city_list.append(city)
            for a_i in range(0, len(china_location_data[p_i]['city'][c_i]['area'])):
                area = remove_p_c_a(china_location_data[p_i]['city'][c_i]['area'][a_i])
                locations_area_list.append(area)

    return locations_province_list, locations_city_list, locations_area_list

# 尝试发现同音地名，写入文件
from pypinyin import lazy_pinyin
def discover_homophone_locations(locations_list):
    homophone_locations_list = []
    for i in range(0, len(locations_list)):
        if i < len(locations_list) - 1:
            for j in range (i + 1, len(locations_list)):
                if lazy_pinyin(locations_list[i]) == lazy_pinyin(locations_list[j]):
                    homophone_locations = locations_list[i] + "\t" + locations_list[j]
                    homophone_locations_list.append(homophone_locations)
        if i % 250 == 0:
            print("%.2f of locations for homophone have processed " %((i/len(locations_list))*100))    
    write_file("common_data/homophone_locations.txt",homophone_locations_list)
    return homophone_locations_list

# 载入并读取地点名称的json文件，返回 provinces_list, citys_list, areas_list
location_file_path = 'common_data/locations.json'   
provinces_list, citys_list, areas_list = read_location(location_file_path)
print(len(provinces_list))
print(len(citys_list))
print(len(areas_list))

# 将所有地名合并到locations里
locations = list(set(provinces_list + citys_list + areas_list))
print(len(locations))
print(locations[1000])

# 尝试发现同音地名
homophone_locations_list = discover_homophone_locations(locations)
print(len(homophone_locations_list))
print(homophone_locations_list)


34
391
3227
3088
阳东
0.00 of locations for homophone have processed 
8.10 of locations for homophone have processed 
16.19 of locations for homophone have processed 
24.29 of locations for homophone have processed 
32.38 of locations for homophone have processed 
40.48 of locations for homophone have processed 
48.58 of locations for homophone have processed 
56.67 of locations for homophone have processed 
64.77 of locations for homophone have processed 
72.86 of locations for homophone have processed 
80.96 of locations for homophone have processed 
89.05 of locations for homophone have processed 
97.15 of locations for homophone have processed 
325
['大观\t大关', '广灵\t广陵', '细河\t西和', '西夏\t西峡', '伊吾\t义乌', '萍乡\t凭祥', '萍乡\t平乡', '拜城\t白城', '肃州\t苏州', '肃州\t宿州', '黔江\t潜江', '淇县\t杞县', '潞西\t泸西', '潞西\t芦溪', '潞西\t泸溪', '睢宁\t遂宁', '睢宁\t绥宁', '富源\t抚远', '鸡西\t绩溪', '延津\t盐津', '香洲\t象州', '浔阳\t旬阳', '龙华\t隆化', '睢阳\t绥阳', '浦城\t蒲城', '平南\t屏南', '玄武\t宣武', '陆丰\t禄丰', '龙南\t陇南', '洛江\t罗江', '简阳\t建阳', '迎江\t盈江', '袁州\t原州', '明山\t名山', 

In [24]:
# 同音地名不作为错误纠正
# 读取同音地名文件，存入list
def read_homophone_locations(file_path):
    homophone_locations_list = []
    text_1, text_2 = read_candidates_test(file_path)
    for i in range(0, len(text_1)):
        homophone_locations_list.append((text_1[i],text_2[i]))
    return homophone_locations_list
    
# 从文件中读取
homophone_locations_list = read_homophone_locations("common_data/homophone_locations.txt")
print(len(homophone_locations_list))
print(homophone_locations_list[0])

# 同音地名不作为错误纠正
exclusion_list = list(set(homophone_locations_list)) 

# 将带有exclusion list中 字词的句子对找出
def discover_exclusion(exclusion_list, text_list_1, text_list_2, text_label_list, label_value = "0", print_progress = False):
    exclusion_text_pair_0_list = []
    exclusion_label_0_list = []
    exclusion_text_pair_1_list = []
    exclusion_label_1_list = []
    count_percent = 0

    print("="*15 + " In Progress " + "="*15)
    for i in range (0, len(text_list_1)):
        count_percent = count_percent + 1
        text_1 = text_list_1[i]
        text_2 = text_list_2[i]
    
        for exclusion in exclusion_list:
            if (len(text_1) == len(text_2)):
                if (exclusion[0] in text_1 and exclusion[1] in text_2) or (exclusion[0] in text_2 and exclusion[1] in text_1):
                    if (text_1.replace(exclusion[0],"") == text_2.replace(exclusion[1],"")) or (text_2.replace(exclusion[0],"") == text_1.replace(exclusion[1],"")):
                        if label_value == "0":
                            exclusion_text_pair_0_list.append(text_list_1[i] + "\t" + text_list_2[i])
                            if text_label_list!= None:
                                exclusion_label_0_list.append(str(i) + "\t" + str(text_label_list[i]))
                            break
                        else:
                            exclusion_text_pair_1_list.append(text_list_1[i] + "\t" + text_list_2[i])
                            if text_label_list!= None:
                                exclusion_label_1_list.append(str(i) + "\t" + str(text_label_list[i]))
                            break

        if print_progress and count_percent%10000 == 0:
            print("%.2f of text pairs for different subsets identification have processed " %((count_percent/len(text_list_1))*100))    
    print("="*15 + "     Done    " + "="*15)
    return exclusion_text_pair_1_list, exclusion_label_1_list, exclusion_text_pair_0_list, exclusion_label_0_list

# 使用 准备数据
test_text_pair_1_list,test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ccf_qianyan_qm_result_B.csv") #仅仅用来查看，不影响结果

exclusion_text_pair_1_list, exclusion_label_1_list, exclusion_text_pair_0_list, exclusion_label_0_list = discover_exclusion(exclusion_list, test_text_pair_1_list,test_text_pair_2_list, view_label_list, "0",  True)
write_file("common_data/exclusion_1_list.txt",exclusion_text_pair_1_list)
write_file("common_data/exclusion_0_list.txt",exclusion_text_pair_0_list)

#查看文件中的句子对数量
print(len(read_candidates_line("common_data/exclusion_1_list.txt")))
print_examples(exclusion_text_pair_1_list,exclusion_label_1_list,None, 100)
print(len(read_candidates_line("common_data/exclusion_0_list.txt")))
print_examples(exclusion_text_pair_0_list,exclusion_label_0_list,None, 100)


The number of lines in common_data/homophone_locations.txt is 325
325
('大观', '大关')

The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ccf_qianyan_qm_result_B.csv is 100000
=============== In Progress ===============
10.00 of text pairs for different subsets identification have processed 
20.00 of text pairs for different subsets identification have processed 
30.00 of text pairs for different subsets identification have processed 
40.00 of text pairs for different subsets identification have processed 
50.00 of text pairs for different subsets identification have processed 
60.00 of text pairs for different subsets identification have processed 
70.00 of text pairs for different subsets identification have processed 
80.00 of text pairs for different subsets identification have processed 
90.00 of text pairs for different subsets identification have processed 
100.00 of text pairs for different subsets identification have


## 3.7 校正Misspelling预测值

In [26]:
# 依据输入的列表校正预测值
def adjust_predict_label(test_a_text_pairs, final_predict_lables, index_and_adjusted_labels, print_correction = True):
    print("test_a_text_pairs: " + str(len(test_a_text_pairs)))
    print("final_predict_lables" + str(len(final_predict_lables)))
    print("index_and_adjusted_labels" + str(len(index_and_adjusted_labels)))
    count = 0
    for i in range(0, len(index_and_adjusted_labels)):
        index_label = index_and_adjusted_labels[i].rstrip().split("\t")
        index = int(index_label[0])
        adjusted_label = index_label[1]
        if final_predict_lables[index]!= adjusted_label:
            count = count + 1
            if print_correction:
                print(test_a_text_pairs[index])
                print('index:' + str(index))
                print('predict_label:' + str(final_predict_lables[index]))
                print('adjusted_label:' + str(adjusted_label))
                print("="*30)
            final_predict_lables[index] = adjusted_label 
    print ("预测值校正个数为: " + str(count))
    return final_predict_lables

In [27]:
# test a text pairs and predict labels
test_a_data_line_list = read_candidates_line("work/test_B_processed.txt")
print("test_a_data_line_list: " + str(len(test_a_data_line_list)))

predict_label_list = read_labels("predict_results/ccf_qianyan_qm_result_B.csv")
print("predict_label_list: " + str(len(predict_label_list)))

# homephone + pycorrector 读取句子对
pinyin_1_list = read_candidates_line("common_data/pinyin_1_text.txt")
pinyin_0_list = read_candidates_line("common_data/pinyin_0_text.txt")
pycorrector_1_list = read_candidates_line("common_data/pycorrector_1_list.txt")
structure_1_list = read_candidates_line("common_data/structure_1_list.txt")
name_1_list = read_candidates_line("common_data/name_1_list.txt")
name_0_list = read_candidates_line("common_data/name_0_list.txt")
equal_1_list = read_candidates_line("common_data/equal_1_list.txt")

misspelling_1_list = list(set(pinyin_1_list + pycorrector_1_list + structure_1_list + name_1_list + equal_1_list))
misspelling_0_list = list(set(pinyin_0_list + name_0_list))
exclusion_1_list = read_candidates_line("common_data/exclusion_1_list.txt")
exclusion_0_list = read_candidates_line("common_data/exclusion_0_list.txt")

print(len(misspelling_1_list))
print(len(misspelling_0_list))

# 已校正的句子对在后续工作中无需重复处理
final_misspelling_list = list(set(misspelling_1_list + misspelling_0_list + exclusion_1_list + exclusion_0_list))
write_file("common_data/misspelling_list.txt",final_misspelling_list )

# 赋值
misspelling_1_index_label_list = []
for i in range (0, len(test_a_data_line_list)):
    if test_a_data_line_list[i] in misspelling_1_list:
        misspelling_1_index_label_list.append(str(i)+"\t1")

misspelling_0_index_label_list = []
for i in range (0, len(test_a_data_line_list)):
    if test_a_data_line_list[i] in misspelling_0_list:
        misspelling_0_index_label_list.append(str(i)+"\t0")

exclusion_0_index_label_list = []
for i in range (0, len(test_a_data_line_list)):
    if test_a_data_line_list[i] in exclusion_0_list:
        exclusion_0_index_label_list.append(str(i)+"\t0")

exclusion_1_index_label_list = []
for i in range (0, len(test_a_data_line_list)):
    if test_a_data_line_list[i] in exclusion_1_list:
        exclusion_1_index_label_list.append(str(i)+"\t1")

# 先后顺序很重要
misspelling_index_label_list =  misspelling_0_index_label_list + misspelling_1_index_label_list + exclusion_0_index_label_list + exclusion_1_index_label_list

print("misspelling_index_label_list: " + str(len(misspelling_index_label_list)))

# 使用 方法4-1 来 校正 预测值 
final_predict_lables = adjust_predict_label(test_a_data_line_list, predict_label_list, misspelling_index_label_list, True)

# 写入文件
write_file("predict_results/ms_ccf_qianyan_qm_result_B.csv",final_predict_lables)

test_a_data_line_list: 100000

The number of predict result lines in predict_results/ccf_qianyan_qm_result_B.csv is 100000
predict_label_list: 100000
6417
403
misspelling_index_label_list: 7650
test_a_text_pairs: 100000
final_predict_lables100000
index_and_adjusted_labels7650
章子怡是那里人	张子怡是那里人
index:2284
predict_label:1
adjusted_label:0
王子怡是谁	汪子怡是谁
index:4449
predict_label:1
adjusted_label:0
乌兰图雅的老公是谁	乌蓝图雅的老公是谁
index:6510
predict_label:1
adjusted_label:0
伊可尔治疗扁平疣	伊可儿治疗扁平疣
index:6982
predict_label:1
adjusted_label:0
程咬金是什么生肖	程咬何是什么生肖
index:14676
predict_label:1
adjusted_label:0
刘亚的拼音	刘娅的拼音
index:19050
predict_label:1
adjusted_label:0
大空翼是哪个球队的	大平翼是哪个球队的
index:19052
predict_label:1
adjusted_label:0
刘晓燕哪里人	刘小燕哪里人
index:21532
predict_label:1
adjusted_label:0
什么玛丽亚	什么玛利亚
index:27038
predict_label:1
adjusted_label:0
武则天原籍是哪里	常则天原籍是哪里
index:30955
predict_label:1
adjusted_label:0
刘恺威多少岁数	刘凯威多少岁数
index:32870
predict_label:1
adjusted_label:0
金三顺作品	金叁顺作品
index:33513
predict_label:1
adjusted_label:0

## 3.8 对比校正结果

In [28]:
#读取test和两个预测值
test_text_list_1, test_text_list_2 = read_candidates_test("work/test_B_1118.tsv")
test_predict_label_list_1 = read_labels("predict_results/ccf_qianyan_qm_result_B.csv")
test_predict_label_list_2 = read_labels("predict_results/ms_ccf_qianyan_qm_result_B.csv")
print(len(test_text_list_1))
print(len(test_text_list_2))
print(len(test_predict_label_list_1))
print(len(test_predict_label_list_2))

# 对比两组预测值，并输出不同的预测结果
def compre_predict_results(text_list1, text_list2, candidates_1, candidates_2):
    list1 = []
    list2 = []
    label1 = []
    label2 = []
    j = 0
    for i in range (0,len(candidates_1)): 
        if candidates_1[i] != candidates_2[i]:
            print("No. " + str(i) + " label 1 = " + str(candidates_1[i]) + " lable 2 = " + str(candidates_2[i]) + " text pair: " + text_list1[i] + " " + text_list2[i])
            list1.append(text_list1[i])
            list2.append(text_list2[i])
            label1.append(candidates_1[i])
            label2.append(candidates_2[i])
            j = j + 1
    print("Total number of differences is " + str(j))
    print("Two prediction's similarity is %.2f"%((len(candidates_2) - j)/len(candidates_2)*100))
    return list1, list2, label1, label2

text_list1, text_list2, correct_text_list1, predict_text_list1= compre_predict_results(test_text_list_1, test_text_list_2, test_predict_label_list_1, test_predict_label_list_2)
print("不同的预测值共有： " + str(len(predict_text_list1)) + " 组")



The number of lines in work/test_B_1118.tsv is 100000

The number of predict result lines in predict_results/ccf_qianyan_qm_result_B.csv is 100000

The number of predict result lines in predict_results/ms_ccf_qianyan_qm_result_B.csv is 100000
100000
100000
100000
100000
No. 1111 label 1 = 0 lable 2 = 1 text pair: 你知道是吗 你知道是
No. 1297 label 1 = 0 lable 2 = 1 text pair: AR镀膜玻璃的介绍 ar镀膜玻璃的
No. 1780 label 1 = 0 lable 2 = 1 text pair: 吃什么养胃 吃十么养胃
No. 2523 label 1 = 0 lable 2 = 1 text pair: 我要运动吗 我要运动
No. 4164 label 1 = 0 lable 2 = 1 text pair: OPPOWATCH防水吗 OPPOWATCH防水
No. 6908 label 1 = 0 lable 2 = 1 text pair: 大连市大学排名 大连大学排名
No. 7671 label 1 = 0 lable 2 = 1 text pair: 呵能组什么词 能组什么词
No. 8119 label 1 = 0 lable 2 = 1 text pair: 主动的动字怎么写 自动的动字怎么写
No. 13024 label 1 = 0 lable 2 = 1 text pair: 月经来之前乳房疼怎么回事 月经来之前乳房痒怎么回事
No. 13355 label 1 = 0 lable 2 = 1 text pair: 药品怎么读 药品怎么卖
No. 13375 label 1 = 0 lable 2 = 1 text pair: 咱们聪明吗 咱们聪明啊
No. 14676 label 1 = 1 lable 2 = 0 text pair: 程咬金是什么生肖 程咬何是什么生肖
No. 1